In [26]:
# import models
import pandas as pd
import numpy as np

#pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 

#models
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [27]:
from project_fraud.data import cleaned_featured_data

**IMPORT DATA**

In [28]:
df = cleaned_featured_data('~/data/')

In [36]:
X = df[['TransactionID','P_emaildomain_suffix','P_emaildomain_bin',
'card1','card2','addr1','TransactionAmt','card5','D15','C13','D2','D10','D4','weekday','hours',\
           'dist_mean', 'dist_median','dist_mean_rel','dist_median_rel']]
y = df['isFraud']

In [47]:
X.shape

(590540, 19)

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3)

sample_size = 1000

X_small = X_train.sample(sample_size, random_state=0)

y_small = y_train.sample(sample_size, random_state=0)

X_train_small, X_test_small, y_train_small, y_test_small = train_test_split(X_small, y_small, random_state=0)

In [48]:
X_small.shape

(1000, 19)

**PIPELINES**

In [38]:
n = (X.dtypes != 'object')
num_cols = list(n[n].index)
medium_missing_num_cols = []
low_missing_num_cols =[]
for i in num_cols:
    percentage = df[i].isnull().sum() * 100 / len(df[i])
    if percentage < 15:
        low_missing_num_cols.append(i)
    elif percentage >= 15 and percentage <= 60:
        medium_missing_num_cols.append(i)

In [39]:
num_transformer_low = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

num_transformer_medium = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant')),
    ('one_hot', OneHotEncoder())
])
    
preprocessor = ColumnTransformer([
    ('low_num_imputer',num_transformer_low, low_missing_num_cols),
    ('medium_num_imputer', num_transformer_medium, medium_missing_num_cols),
    ('cat_transformer', cat_pipeline, ['P_emaildomain_suffix','P_emaildomain_bin','weekday','hours'])
],remainder='drop')


**MODEL RANDOM FOREST CLASSIFIER**

In [40]:
pipeline_rfc = Pipeline([
    ("preprocessor", preprocessor),
    ("rfc", RandomForestClassifier(n_estimators=1500, max_features='auto', max_depth = 50)),
     ])

In [41]:
pipeline_rfc.fit(X_train_small, y_train_small)

Pipeline(memory=None,
     steps=[('preprocessor', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('low_num_imputer', Pipeline(memory=None,
     steps=[('imputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
  ...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [46]:
y_pred_rfc = pipeline_rfc.predict(X_test_small)
y_pred_rfc

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [52]:
pipeline_rfc.score(X_test)

ValueError: Found unknown categories ['uk', 'gmail', 'es', 'jp', 'de'] in column 0 during transform

In [44]:
from sklearn.metrics import f1_score
f1_rfc = f1_score(y_test,y_pred_rfc)
f1_rfc

ValueError: Found input variables with inconsistent numbers of samples: [177162, 250]

In [49]:
from sklearn.metrics import recall_score
recall_rfc = recall_score(y_test,y_pred_rfc)
recall_rfc

ValueError: Found input variables with inconsistent numbers of samples: [177162, 250]

In [ ]:
random_search.best_score_

In [ ]:
random_search.best_estimator_

In [ ]:
random_search.best_params_

What is the performance of the optimal pipeline? Make sure you cross validate!

In [ ]:
from sklearn.model_selection import cross_val_score

# Cross validate optimal pipeline
cross_val_score(random_search.best_estimator_, 
                data.drop(columns="malignant"), data['malignant'],
                cv=10,
                scoring = ['f1', 'recall']).mean()

In [ ]:
from sklearn.model_selection import cross_val_predict

In [ ]:
y_pred_2 = cross_val_predict(random_search.best_estimator_, X, y, cv=5)